In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq

model = ChatGroq(model="openai/gpt-oss-120b", temperature=0)

In [3]:
import sys
import asyncio

# Fix for Windows issues in Jupyter notebooks
if sys.platform == "win32":
    # 1. Use ProactorEventLoop for subprocess support
    if not isinstance(asyncio.get_event_loop_policy(), asyncio.WindowsProactorEventLoopPolicy):
        asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    
    # 2. Redirect stderr to avoid fileno() error when launching MCP servers
    if "ipykernel" in sys.modules:
        sys.stderr = sys.__stderr__


## Local MCP server

In [4]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "local_server": {
                "transport": "stdio",
                "command": "python",
                "args": ["resources/2.1_mcp_server.py"],
            }
    }
)

In [5]:
# get tools
tools = await client.get_tools()

# get resources
resources = await client.get_resources("local_server")

# get prompts
prompt = await client.get_prompt("local_server", "prompt")
prompt = prompt[0].content

In [6]:
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=prompt
)

In [7]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Tell me about the langchain-mcp-adapters library")]},
    config=config
)

In [8]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Tell me about the langchain-mcp-adapters library', additional_kwargs={}, response_metadata={}, id='b024f377-5df0-405b-afd6-c4b5e5f6b32a'),
              AIMessage(content='', additional_kwargs={'reasoning_content': "We need to answer about langchain-mcp-adapters library. Likely a library related to LangChain? Let's search web.", 'tool_calls': [{'id': 'fc_0751696a-612c-478b-908b-b740417fd188', 'function': {'arguments': '{"query":"langchain-mcp-adapters library"}', 'name': 'search_web'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 266, 'total_tokens': 326, 'completion_time': 0.12666639, 'completion_tokens_details': {'reasoning_tokens': 27}, 'prompt_time': 0.011611662, 'prompt_tokens_details': None, 'queue_time': 0.005009331, 'total_time': 0.138278052}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_7ee00986fe', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs':

In [9]:
print(response["messages"][-1].content)

**langchain‑mcp‑adapters** is a small, officially‑maintained library that bridges **Model Context Protocol (MCP)** tool servers with **LangChain** (and, by extension, LangGraph).  

---

## What problem does it solve?

* **MCP** is an open protocol that lets a server expose “tools” (functions, APIs, local subprocesses, etc.) to LLMs in a standard way.  
* LangChain agents expect tools to be instances of `BaseTool` (Python) or `Tool` (JS/TS).  
* Writing a custom wrapper for every MCP server quickly becomes tedious.

`langchain‑mcp‑adapters` automates that conversion:

| MCP side | LangChain side |
|----------|----------------|
| `tool` defined in an MCP server (JSON‑RPC over HTTP, WebSocket, stdio, etc.) | `BaseTool` / `Tool` object that can be passed to `AgentExecutor`, `createAgent`, `Graph`, … |
| Multiple MCP servers | One client (`MultiServerMCPClient`) that aggregates all tools and presents them as a single list. |
| OAuth‑2 / API‑key protected MCP servers | Built‑in `authProvide

## Online MCP

In [10]:
client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "uvx",
            "args": [
                "mcp-server-time",
                "--local-timezone=America/New_York"
            ]
        }
    }
)

tools = await client.get_tools()

In [11]:
agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=prompt
)

In [12]:
question = HumanMessage(content="What time is it?")

response = await agent.ainvoke(
    {"messages": [question]}
)

pprint(response)

{'messages': [HumanMessage(content='What time is it?', additional_kwargs={}, response_metadata={}, id='8b7cc6ee-3ffe-4cf8-9e0f-0c6c79b282f9'),
              AIMessage(content="I'm sorry, I can only answer questions about LangChain, LangGraph and LangSmith.", additional_kwargs={'reasoning_content': 'User asks "What time is it?" Not related to LangChain etc. According to instructions, we must respond with "I\'m sorry, I can only answer questions about LangChain, LangGraph and LangSmith."'}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 426, 'total_tokens': 496, 'completion_time': 0.147075274, 'completion_tokens_details': {'reasoning_tokens': 43}, 'prompt_time': 0.020547354, 'prompt_tokens_details': None, 'queue_time': 0.004762654, 'total_time': 0.167622628}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_8634b911ff', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c52d2-57f1-7

In [13]:
print(response["messages"][-1].content)

I'm sorry, I can only answer questions about LangChain, LangGraph and LangSmith.
